In [16]:
import scanpy as sc
import multigrate as mtg

In [17]:
rna = sc.read('/lustre/groups/ml01/projects/2022_multigrate_anastasia.litinetskaya/multimil_reproducibility/pipeline/data/pp/pbmc_healthy_covid_rna.h5ad')
rna

AnnData object with n_obs × n_vars = 624325 × 2000
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id', 'split0', 'split1', 'split2', 'split3', 'split4'
    var: 'feature_types', 'means', 'variances', 'residual_variances', 'highly_variable_rank', 'highly_variable_nbatches', 'highly_variable_intersection', 'highly_variable'
    uns: 'hvg', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'counts'

In [3]:
rna.X.data

array([1., 9., 2., ..., 7., 3., 7.], dtype=float32)

In [4]:
adt = sc.read('../../../pipeline/data/pp/pbmc_healthy_covid_adt.h5ad')
adt

AnnData object with n_obs × n_vars = 624325 × 192
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id', 'split0', 'split1', 'split2', 'split3', 'split4'
    var: 'feature_types'
    uns: 'hvg', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    layers: 'counts'

In [5]:
adt.X.data

array([0.16182648, 0.4232845 , 0.80181926, ..., 1.3309513 , 1.3929683 ,
       1.9186018 ], dtype=float32)

In [6]:
adata = mtg.data.organize_multiome_anndatas(
     adatas = [[rna], [adt]],
)
adata

AnnData object with n_obs × n_vars = 624325 × 2192
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id', 'split0', 'split1', 'split2', 'split3', 'split4', 'group'
    var: 'modality'
    uns: 'modality_lengths'
    layers: 'counts'

In [7]:
mtg.model.MultiVAE.setup_anndata(
    adata,
    rna_indices_end=2000,
    categorical_covariate_keys=[
        "Site",
        "patient_id",
      ])

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multimil/lib/python3.10/site-packages/scvi/data/fields/_layer_field.py:90: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(


In [8]:
vae = mtg.model.MultiVAE(
    adata, 
    losses=[
        "nb", "mse"
    ],
    loss_coefs={
        "kl": 0.01,
    },
    cond_dim=16,
    z_dim=16,
)

In [ ]:
vae.train(lr=1e-4)

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multimil/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.10 /lustre/groups/ml01/workspace/anastasia.litinets ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multimil/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.10 /lustre/groups/ml01/workspace/anastasia.litinets ..

Epoch 125/200:  62%|██████▏   | 124/200 [1:02:12<29:43, 23.47s/it, loss=830, v_num=1] 

In [ ]:
vae.plot_losses()

In [ ]:
vae.get_latent_representation()
adata

In [ ]:
sc.pp.neighbors(adata, use_rep='latent')
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(
    adata, 
    color=[
      "Status_on_day_collection_summary",
      "Site",
      "initial_clustering"
    ],
    ncols=1,
    frameon=False
)

In [ ]:
adata.write('../../../pipeline/data/pp/pbmc_healthy_covid_multigrate.h5ad')

In [ ]:
vae.save('model/', overwrite=True)